In [2]:
import pandas as pd

# Load data

## Graph0 - UserID

In [3]:
userID_accountID = pd.read_csv('raw/userID_accountID.csv')
userID_cardID = pd.read_csv('raw/userID_cardID.csv')
userID_deviceID = pd.read_csv('raw/userid_deviceid.csv')

In [5]:
total_users = list(set(userID_accountID['userID'].tolist() + (userID_cardID['userID'].tolist()) + (userID_deviceID['userID'].tolist())))
total_users[:10]

[191104935002113,
 200513000046595,
 191223024058373,
 191129565003782,
 200516000022534,
 191012848009224,
 200125000974345,
 191016550006794,
 200123000029195,
 200615000014857]

## Old TPE, device_user

In [9]:
android_df = pd.read_csv('raw/dataTest_device_profile_android.csv')
ios_df = pd.read_csv('raw/dataTest_device_profile.csv')

joint_df = pd.concat([android_df, ios_df])

In [10]:
joint_df.head()

,timestamp,userId,params
0,1596244856,170425000002238,"{""hw_released"":""Tue Jul 07 08:50:39 GMT+07:00 ..."
1,1596244830,191113586003363,"{""hw_released"":""Thu Jul 02 12:19:14 GMT+07:00 ..."
2,1596244888,200413000001404,"{""hw_released"":""Thu Dec 07 11:24:05 GMT+07:00 ..."
3,1596244831,200428000037320,"{""hw_released"":""Wed Apr 22 07:53:33 GMT+07:00 ..."
4,1596244849,190309000002524,"{""hw_released"":""Thu Mar 12 12:05:05 GMT+07:00 ..."


In [11]:
device_profile_user_list = joint_df['userId'].tolist()

In [13]:
device_profile_user_list[:10]

[170425000002238,
 191113586003363,
 200413000001404,
 200428000037320,
 190309000002524,
 190104000013363,
 180201000023650,
 200712000080516,
 200801000006844,
 191112542003496]

In [15]:
columns = ['deviceID', 'transID', 'transType', 'appid', 'pmcID', 'amount', 'userID', 'osVer', 'deviceModel', 'userIP', 'reqDate', 'longitude', 'latitude', 'campaignID', 'type_transaction']
tpe = pd.read_csv('raw/tpe_device_score.csv', names=columns)
tpe.head()

,deviceID,transID,transType,appid,pmcID,amount,userID,osVer,deviceModel,userIP,reqDate,longitude,latitude,campaignID,type_transaction
0,000195863904dc8a,200808000071150,1,454,39,2000000,200119000040347,Android 28 (9),Samsung SM-J730G,125.214.48.174,2020-08-08 08:51:08.856,105.952103,10.223810,0,payment
1,000195863904dc8a,200808000089357,1,454,39,2000000,200119000040347,Android 28 (9),Samsung SM-J730G,125.214.48.174,2020-08-08 09:47:11.279,105.952107,10.223811,0,payment
2,000195863904dc8a,200809000160928,1,454,39,2000000,200119000040347,Android 28 (9),Samsung SM-J730G,125.214.48.174,2020-08-09 14:43:45.359,105.952099,10.223812,0,payment
3,000195863904dc8a,200822000188187,1,454,39,2000000,200119000040347,Android 28 (9),Samsung SM-J730G,125.214.48.174,2020-08-22 14:33:30.566,105.952106,10.223818,0,payment
4,000195863904dc8a,200822000271777,1,454,39,2000000,200119000040347,Android 28 (9),Samsung SM-J730G,125.214.48.174,2020-08-22 20:30:36.476,105.952090,10.223808,0,payment


In [16]:
tpe_user_list = tpe['userID'].tolist()
tpe_user_list[:10]

[200119000040347,
 200119000040347,
 200119000040347,
 200119000040347,
 200119000040347,
 200119000040347,
 200119000040347,
 200119000040347,
 200119000040347,
 200119000040347]

## Bank Linking

In [17]:
bank = pd.read_csv('raw/bankling.csv')
bank.head()

,userID,reqDate,requestStatus,accountId,bankname,cardId,bankCode
0,160516000000516,1591950265034,1,NaN,CC,2.006120e+14,ZPCC
1,160516000000516,1596798253563,1,NaN,CC,2.006120e+14,ZPCC
2,160919000012009,1580263971856,1,2.001291e+14,MSB,NaN,ZPMSB
3,160922000002026,1579068114314,1,NaN,CC,2.001150e+14,ZPCC
4,161029000000001,1596605415399,-10335,NaN,TPB,2.008050e+14,ZPTPB


In [19]:
bank_user_list = bank['userID'].tolist()
bank_user_list[:10]

[160516000000516,
 160516000000516,
 160919000012009,
 160922000002026,
 161029000000001,
 161223000000016,
 161223000000016,
 170112000004362,
 170112000004362,
 170124000004114]

## Select more from userid

In [ ]:
aql1 = """
FOR v IN userid
    SORT v.reqDate
    LIMIT 1000000
    RETURN v._key
"""
queryResult = db.AQLQuery(aql1, rawResults=True,batchSize=4000)

In [ ]:
len(queryResult)

In [ ]:
to_save = []
for x in queryResult:
    to_save.append(x)

In [ ]:
len(to_save)

# Process

## Join all user list

In [20]:
users = list(set(total_users + device_profile_user_list + tpe_user_list + bank_user_list))

In [22]:
user_df = pd.DataFrame(users, columns=["userID"])

In [24]:
user_df.to_csv('users.csv', index=False)

In [30]:
user_df.head()

,userID
0,180921000001536
1,200830000037890
2,200125000450052
3,180224000000004
4,180224000000006


## Query related edges

## Import libraries and data

In [6]:
from pyArango.connection import *
import pandas as pd

In [4]:
user_df = pd.read_csv('./users.csv')

In [5]:
user_df.head()

,userID
0,180224000000004
1,180224000000006
2,200803000254471
3,170405000000077
4,181115000012471


## Format to get alike document structure

In [12]:
user_df = pd.read_csv('./users.csv')
user_df.head()

,userID,userID_format
0,180224000000004,userid/180224000000004
1,180224000000006,userid/180224000000006
2,200803000254471,userid/200803000254471
3,170405000000077,userid/170405000000077
4,181115000012471,userid/181115000012471


In [10]:
user_list = user_df['userID_format'].tolist()
user_list[:10]

['userid/180224000000004',
 'userid/180224000000006',
 'userid/200803000254471',
 'userid/170405000000077',
 'userid/181115000012471',
 'userid/180723000001184',
 'userid/191022416003087',
 'userid/180224000000017',
 'userid/180224000000018',
 'userid/181230000005660']

## Connect to ArangoDB using pyArango

In [32]:
conn = Connection(arangoURL="http://10.40.38.4:8529",username="baonlc", password="user_df")

In [33]:
db = conn['user_network']

In [47]:
aql = f"""
LET idList = @user_list
FOR id IN idList
    FOR v, e IN 1..1 ANY id GRAPH "transaction"
        RETURN e
"""
queryResult = db.AQLQuery(aql, rawResults=True, bindVars={user_list: user_list},batchSize=200)

In [ ]:
result = []
for qr in queryResult:
    result.append(qr)

In [ ]:
result_df = pd.DataFrame(result)

In [ ]:
result_df.to_csv('./transaction.csv', index=False)